# SageMaker で自分たちの機械学習アルゴリズムの学習・推論を行う

#### ノートブックに含まれる内容

- BYOA を SageMaker で行うときの，基本的なやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: DecisionTreeClassifier
- データ: iris

## セットアップ

In [ ]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

# AWS credential で指定された role を返す
role = get_execution_role()

In [ ]:
# SageMaker のセッションを作成
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## 学習用データを S3 にアップロード

SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，ローカルにある iris データをいったん SageMaker SDK の `session` クラスにある `upload_data()` メソッドを使って，ノートブックインスタンスのローカルから S3 にアップロードします．

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．`upload_data()` メソッドの引数に `bucket=XXXX` という形でデータを配置するバケットを指定することが可能です．

以下を実行する前に，**<span style="color: red;">2 行目の `data/scikit-byo-iris/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
# S3 プレフィックス
prefix = 'data/scikit-byo-iris/XX'

WORK_DIRECTORY = 'data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## モデルの学習を実行

SageMaker で学習を行うために，SageMaker SDK で `Estimator` オブジェクトをつくります．このオブジェクトには，学習をおこなうために以下の設定が含まれます．その上で，`fit()` メソッドで学習を実施します．学習には 5 分程度時間がかかります．

* __container name__: 02 のノートブックで作成した ECR のコンテナイメージ
* __role__: ジョブを実行する IAM role
* __instance count__:  学習ジョブに使うインスタンス数
* __instance type__ 学習ジョブに使うインスタンスタイプ
* __output path__: 学習の成果物が置かれる S3 の場所
* __session__: すぐ上で作成した，SageMaker セッション

また，以下を実行する前に，**<span style="color: red;">3 行目の `decision-trees-sample-XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/decision-trees-sample-XX'.format(account, region)

classifier = sage.estimator.Estimator(
    image,
    role,
    1,
    'ml.m4.xlarge',
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess)

classifier.fit(data_location)

また，学習のジョブを実行する際に，ハイパーパラメタを変えて複数のジョブを実行したい場合があります．現在の SageMaker Python SDK の　`Estimato.fit()` メソッドは[非同期実行をサポートしていない](https://github.com/aws/sagemaker-python-sdk/blob/f43b9c3c54b152829e7b683c5a46e4cd2b3d242e/src/sagemaker/estimator.py#L156)ため，ここでは簡単な[ラッパー](util/estimator_wrapper.py)を作って，非同期で実行できるようにしています（なお，[Issue は既に上がっている](https://github.com/aws/sagemaker-python-sdk/issues/4)ため，じきに修正される予定です）．

In [ ]:
from util.estimator_wrapper import EstimatorWrapper

wrapper = EstimatorWrapper(classifier)

wrapper.set_hyperparameters(max_leaf_nodes=10)
wrapper.fit(data_location)
wrapper.set_hyperparameters(max_leaf_nodes=5)
wrapper.fit(data_location)

## モデルの推論を実行

推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．また併せて，オプションで（リクエストで渡されるデータの）シリアライザと（レスポンスで返されるデータの）デシリアライザを指定することも可能です．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = classifier.deploy(
    1,
    'ml.m4.xlarge',
    serializer=csv_serializer)

In [ ]:
# 推論用のデータを準備
test_X = pd.DataFrame([[5.0, 3.2, 1.2, 4.3], [4.5, 2.3, 1.3, 0.3], [5.7, 2.8, 4.1, 1.3]])

print(test_X)

In [ ]:
# 推論を実行して，結果を表示
for i, row in test_X.iterrows():
    result = predictor.predict(row.values).decode('utf-8')
    print('predict result of data[{}] is "{}"'.format(i, result.rstrip()))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sess.delete_endpoint(predictor.endpoint)